In [2]:
# check overlap with EXO-21-008 signal

import ROOT as rt
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward

import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import  make_datacard_2sig, make_datacard_2tag, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaR

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0


print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [3]:
fpath =OrderedDict()
tree = OrderedDict()
mass = [15, 40, 55]

# prod = ['ggH', 'VBFH_H','ZH', 'WH', 'ttH_H','ggZH']
# decay = 'dddd'


prod = ['ggH','VBFH','ZH', 'WH', 'ttH_H','ggZH']
prod = ['ggH']
decay = '4Tau'

prod = ['ggH','VBFH','ZH', 'WH', 'ttH_H','ggZH']
decay = 'bbbb'


mass = [15, 40, 55]
if not decay == 'bbbb': mass = [7, 15, 40, 55]

OLD_CTAU = np.array([100, 1000, 10000, 100000])#in mm

ntupler_version = 'V1p17/'

# data_path = '/storage/cms/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018/v5/v100/normalized/' #what is used in final version of EXO-20-015
data_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//Data2018//v5/v138//normalized/'
fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'
mc_path = {}

analyzer_version = 'v1/v138/'
mc_path['ggH'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
mc_path['VBFH'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
analyzer_version = 'v2/v138/'
mc_path['ZH'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['WH'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['WminusH'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['WplusH'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['ttH_H'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['VBFH_H'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['ggZH'] = mc_path['ZH']




for m in mass:
    for ct in OLD_CTAU:
        for p in prod:
            if p == 'others':continue
            if p == 'VBFH' and decay == 'bbbb':
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)                       
                fpath[key] = mc_path[p]+p+'_HToSSTo4b_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_137000pb_weighted.root'
            elif p == 'ggH' or p == 'VBFH':
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)                       
                fpath[key] = mc_path[p]+p+'_HToSSTo'+decay+'_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_137000pb_weighted.root'
            elif p == 'VBFH_H':
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)  
                fpath[key] = mc_path[p]+'VBFHToSS_STodd_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
            else:
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)                       
                if decay == 'bbbb':fpath[key] = mc_path[p] + p+'ToSS_SToBB_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                if decay == 'dddd':fpath[key] = mc_path[p] + p+'ToSS_STodd_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                if decay == '4Tau':fpath[key] = mc_path[p] + p+'ToSS_SToTauTau_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                if p == 'ggZH':
                    if decay == 'bbbb':fpath[key] = mc_path[p] + 'ZHToSS_SToBB_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                    if decay == 'dddd':fpath[key] = mc_path[p] + 'ZHToSS_STodd_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                    if decay == '4Tau':fpath[key] = mc_path[p] + 'ZHToSS_SToTauTau_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'


# fpath['hnl']= '/storage/af/user/christiw/HNL_electronType_ms5p0_plVe2_1e-5/HeavyNeutralLepton_Tree.root'                        
NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    
    w = tree[k]["weight"].array()
    if not 'data' in k: 
        print(k, root_dir['NEvents']._fEntries)
        


root_dir = uproot.open('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root') 
h_reweight = root_dir['higgsPthiggsEta']

MC_ggH_15_100 6984998.0
MC_VBFH_15_100 2425654.0
MC_ZH_15_100 199500.0
MC_WH_15_100 297997.0
MC_ttH_H_15_100 102500.0
MC_ggZH_15_100 199500.0
MC_ggH_15_1000 6924000.0
MC_VBFH_15_1000 2350380.0
MC_ZH_15_1000 149900.0
MC_WH_15_1000 299500.0
MC_ttH_H_15_1000 130000.0
MC_ggZH_15_1000 149900.0
MC_ggH_15_10000 7157000.0
MC_VBFH_15_10000 856280.0
MC_ZH_15_10000 149297.0
MC_WH_15_10000 377298.0
MC_ttH_H_15_10000 149400.0
MC_ggZH_15_10000 149297.0
MC_ggH_15_100000 7075998.0
MC_VBFH_15_100000 892602.0
MC_ZH_15_100000 768698.0
MC_WH_15_100000 1210293.0
MC_ttH_H_15_100000 178900.0
MC_ggZH_15_100000 768698.0
MC_ggH_40_100 6931381.0
MC_VBFH_40_100 813041.0
MC_ZH_40_100 1047369.0
MC_WH_40_100 2173016.0
MC_ttH_H_40_100 50000.0
MC_ggZH_40_100 1047369.0
MC_ggH_40_1000 6916000.0
MC_VBFH_40_1000 832573.0
MC_ZH_40_1000 149490.0
MC_WH_40_1000 298386.0
MC_ttH_H_40_1000 149300.0
MC_ggZH_40_1000 149490.0
MC_ggH_40_10000 7056998.0
MC_VBFH_40_10000 825334.0
MC_ZH_40_10000 148492.0
MC_WH_40_10000 248086.0
MC_ttH_

# load branches

In [4]:

JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
ring_cut = 50
cut_based = True
cut_based_version = 'v4'

intime = True
DPHI_CUT = 1
weight = {}

nCsc_JetMuonVetoCluster0p4_Me1112Veto = {}
dphiMet_cluster = {}

cscRechitClusterEta = {}
cscRechitClusterNStation10  = {}
cscRechitClusterAvgStation10 = {}

sel_ev = {}
bdt_sel = {}
sel_csc = {}
sel_dt = {}

ggZH_weight = {}
higgsEta = {}
higgsPt = {}


cluster_index = ''

NStation_weight = {}
AvgStation_weight = {}
Eta_weight = {}

sel_csccluster = {}
sel_dtcluster = {}
dt_dt = {}
csc_csc = {}
csc_dt = {}
nDtWheels25 = {}
nDtStations25 = {}
two_tag = {}
dR = {}
for k in list(tree.keys()):
    if 'data' in k:continue
#     if not k == 'data_intime_sr':continue
########### SELECTION: CLUSTERS ############
    if 'data' in k: T = tree['data']
    else: T = tree[k]

#     if '15' in k or '40' in k:continue
#     if not k == 'old':continue
########### SELECTION: CLUSTERS ############


    sel_rechitcluster = np.logical_and( T.array('dtRechitClusterMuonVetoPt') < 10, T.array('dtRechitClusterJetVetoPt') < 20)
     
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_RPChits_dPhi0p5') > 0)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_minus') <= 8)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_plus') <= 8)  
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_0p5') <= 1)


########### SELECTION: JETS ############
    

    sel_jet = np.logical_and(T.array('jetPt') > 30, np.abs(T.array('jetEta')) < 2.4 )
    sel_jet = np.logical_and(T.array('jetTightPassId'), sel_jet)
########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()
    sel_ev[k] = T.array('metEENoise') > 200
    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)

    sel_ev[k] = np.logical_and(sel_ev[k],T.array('nDtStations25')<3)
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('nDtWheels25')<3)
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
    sel_ev[k] = np.logical_and(sel_ev[k],np.abs(T.array('jetMet_dPhiMin'))>0.6)

    sel_ev[k]  = np.logical_and(sel_ev[k],sel_rechitcluster.sum() >= 1)



########### BRANCHES ############

    


    higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]]
    higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]]
    ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]


    if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]]
    else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]]
    if 'ggZH' in k: weight[k] *= ggZH_weight[k]
        
    ###################
    # 2 tag selections
    ###################
    sel_csccluster[k] = np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < 12.5,  T.array('cscRechitCluster' + cluster_index + 'TimeTotal') > -5)
    sel_csccluster[k] = np.logical_and(sel_csccluster[k], np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<1.2)
    sel_csccluster[k] = np.logical_and(sel_csccluster[k], T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')<30)
    sel_csccluster[k] = np.logical_and(sel_csccluster[k],  T.array('cscRechitCluster' + cluster_index + 'TimeSpread') <= 20)
    sel_csccluster[k] = np.logical_and(sel_csccluster[k],  T.array('cscRechitCluster' + cluster_index + 'Me11Ratio') <1)


    sel_csccluster[k] = sel_csccluster[k][sel_ev[k]]
    sel_dtcluster[k] = T.array('dtRechitCluster_match_RPChits_dPhi0p5')>0
    sel_dtcluster[k] = np.logical_and(sel_dtcluster[k], np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1)
    sel_dtcluster[k] = np.logical_and(sel_dtcluster[k], np.abs(T.array('dtRechitClusterJetVetoPt')) < 50)
    sel_dtcluster[k] = np.logical_and(sel_dtcluster[k], np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))
    sel_dtcluster[k] = sel_dtcluster[k][sel_ev[k]]

    #########################
    # overlap calculation
    #########################

    csc_csc[k] = np.logical_and(sel_csccluster[k].sum() == 2, sel_dtcluster[k].sum()==0)
    dt_dt[k] = np.logical_and(sel_csccluster[k].sum() == 0, sel_dtcluster[k].sum()==2)
    csc_dt[k] = np.logical_and(sel_csccluster[k].sum() == 1, sel_dtcluster[k].sum()==1)

    dR[k] = deltaR(T.array('cscRechitClusterEta')[sel_ev[k]][sel_csccluster[k]][csc_dt[k]][:,0], \
               T.array('cscRechitClusterPhi')[sel_ev[k]][sel_csccluster[k]][csc_dt[k]][:,0],\
               T.array('dtRechitClusterEta')[sel_ev[k]][sel_dtcluster[k]][csc_dt[k]][:,0],\
               T.array('dtRechitClusterPhi')[sel_ev[k]][sel_dtcluster[k]][csc_dt[k]][:,0])

    nDtStations25[k] = T.array('nDtStations25')[sel_ev[k]]
    nDtWheels25[k]= T.array('nDtWheels25')[sel_ev[k]]
    two_tag[k] = sel_csccluster[k].sum() + sel_dtcluster[k].sum()>=2

#     print(k, '\t', np.count_nonzero(csc_csc[k])/len(csc_csc[k]))


# data overlap

In [24]:
k = 'data_intime_sr'
print(np.count_nonzero(csc_csc[k]), np.count_nonzero(dt_dt[k]), np.count_nonzero(csc_dt[k]))
print(len(csc_csc[k]), len(dt_dt[k]), len(csc_dt[k]))
cond = sel_csccluster[k].sum()+sel_dtcluster[k].sum()>2

print(sel_csccluster[k].sum()[cond], sel_dtcluster[k].sum()[cond], nDtStations25[k][cond], nDtWheels25[k][cond])
print(np.count_nonzero(bdt_sel[k]))

0 0 1
149 149 149
[] [] [] []
149


# signal overlap

In [6]:
for m in mass:
    for ct in OLD_CTAU:
        dt_analysis = 0
        csc_csc_temp = 0
        csc_dt_temp = 0
        dt_dt_temp = 0
        gt2tag = 0
        for p in prod:
            k = 'MC_'+p+'_'+str(m)+'_'+str(ct)                       
            cond = sel_csccluster[k].sum()+sel_dtcluster[k].sum()>2
            dt_analysis += np.sum(weight[k])
            csc_csc_temp += np.sum(weight[k][ csc_csc[k]])
            csc_dt_temp += np.sum(weight[k][csc_dt[k]])
            dt_dt_temp += np.sum(weight[k][dt_dt[k]])
            gt2tag += np.sum(weight[k][cond])
 
#         print(str(m)+'_'+str(ct), '\t',csc_csc_temp/dt_analysis, \
#               '\t',dt_dt_temp/dt_analysis, \
#               '\t',csc_dt_temp/dt_analysis, \
#               '\t',gt2tag/dt_analysis)
        print(str(m)+' GeV, c$\\tau$ '+str(ct)+' mm', '& ',str(round(csc_csc_temp/dt_analysis*100,2))+'\\%', \
              '& ',str(round(dt_dt_temp/dt_analysis*100,2))+'\\%', \
              '& ',str(round(csc_dt_temp/dt_analysis*100,2))+'\\%', \
              '& ',str(round((csc_csc_temp+dt_dt_temp+csc_dt_temp)/dt_analysis*100,2))+'\\%', '\\\\')



        #     print(len(csc_csc[k]), len(dt_dt[k]), len(csc_dt[k]))
#     print(np.count_nonzero(bdt_sel[k]))

15 GeV, c$\tau$ 100 mm &  0.0\% &  0.0\% &  0.26\% &  0.26\% \\
15 GeV, c$\tau$ 1000 mm &  0.0\% &  4.23\% &  5.37\% &  9.6\% \\
15 GeV, c$\tau$ 10000 mm &  0.0\% &  0.38\% &  0.64\% &  1.02\% \\
15 GeV, c$\tau$ 100000 mm &  0.0\% &  0.0\% &  0.13\% &  0.13\% \\
40 GeV, c$\tau$ 100 mm &  0.0\% &  0.0\% &  0.0\% &  0.0\% \\
40 GeV, c$\tau$ 1000 mm &  0.0\% &  4.68\% &  7.06\% &  11.74\% \\
40 GeV, c$\tau$ 10000 mm &  0.0\% &  1.54\% &  1.66\% &  3.21\% \\
40 GeV, c$\tau$ 100000 mm &  0.0\% &  0.08\% &  0.13\% &  0.21\% \\
55 GeV, c$\tau$ 100 mm &  nan\% &  nan\% &  nan\% &  nan\% \\
55 GeV, c$\tau$ 1000 mm &  0.0\% &  0.41\% &  3.83\% &  4.25\% \\
55 GeV, c$\tau$ 10000 mm &  0.0\% &  0.43\% &  1.12\% &  1.54\% \\
55 GeV, c$\tau$ 100000 mm &  0.0\% &  0.04\% &  0.36\% &  0.4\% \\


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


# signal yield summary table

In [85]:
ctaus = [ '100', '1000','10000','100000' ]


BR = 0.01
N_RECHIT_CUT = 130
DPHI_CUT = 0.75
corrections = 0.8953452999999999

# corrections = 0.902012765
var = dphiMet_cluster
# DPHI_CUT = 0.6      
# var = jetMet_dPhiMin30
for N_RECHIT_CUT in np.arange(60,220,10):
    if not N_RECHIT_CUT==130:continue
    total_sig = 0
    for m in mass:
#         if not m == 55:continue
        signal_rate = []
        unc_rate = []
        signal_unc = []
        for ct in ctaus:   
#             if not ct == '1000':continue
            signal = 0
            sig_unc = 0
            shape_unc_temp = 0
            ctf = int(ct)
            if ctf < OLD_CTAU[0]:
                old_ctau_temp = np.array([OLD_CTAU[0]])
            else:
                for j, ct0 in enumerate(OLD_CTAU):
                    if ct0 == ctf: 
                        old_ctau_temp = np.array([int(ctf)])
                        break

                    elif ct0 > ctf:
                        old_ctau_temp = np.array([OLD_CTAU[j-1], OLD_CTAU[j]])
                        old_ctau_temp = np.array([OLD_CTAU[j]])
                        break
                    if j == len(OLD_CTAU)-1: 
                        old_ctau_temp = np.array([OLD_CTAU[j]])

            weight_sum = 0
            weight_len = 0
            for j,ct0 in enumerate(old_ctau_temp):
                prods = ['ggH', 'VBFH','WH','ZH','ttH_H','ggZH']
                prods = ['VBFH',]
                #VBFH and WH is a bit off
                for p in prods:
                    
                    k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)+''
                    

                    T = tree[k]
                    if np.count_nonzero(sel_ev[k]) == 0: continue
                    gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                    if len(gLLP_ctau) == 0: continue

                    w = weight[k]*AvgStation_weight[k][bdt_sel[k]]*NStation_weight[k][bdt_sel[k]]
                    w = weight[k]
                    # higgs pt, signal uncertainty weight
                    cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, np.abs(var[k])<DPHI_CUT)#bin D
#                     cond = np.logical_and(cond, np.logical_not(two_tag[k]))
                    signal += np.sum(w[cond])
#                     signal += np.count_nonzero(cond)
#                     print(np.mean(w[cond]))
                    sig_unc +=np.sum(w[cond]*w[cond])
#                     shape_unc_temp = (shape_unc_temp**2+np.sum(w[cond]*shape_unc[cond])**2)**0.5


            signal_rate.append(signal)
            signal_unc.append(sig_unc**0.5)
            unc_rate.append(shape_unc_temp)
        signal_rate = np.array(signal_rate)
        unc_rate = np.array(unc_rate)
        signal_unc = np.array(signal_unc)
#         print(N_RECHIT_CUT, '\t', signal_rate[0]*BR)
#         print(m,'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*BR*corrections])),'\\\\')
#         print(m,'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*limit[m]])),'\\\\')

#         print(m, '\t', '\t'.join(map(str, [round(num,2) for num in signal_unc*BR])))
        print(m, '\t', '\t'.join(map(str, [round(num,1) for num in signal_rate*BR*corrections])))


15 	 1.9	6.3	1.5	0.1
40 	 0.0	5.3	4.4	0.7
55 	 0.0	2.2	5.9	0.9


# signal uncertainties

In [86]:
jetVeto = {}
muonVeto = {}
rechitVeto = {}
cut_based_eff_unc = {}
clusterEff_unc = {}
higgsPtWeight = {}
JES = {}
pileup = {}
# ctau_reweight = {}
mc_stats = {}
scaling = {}
lumi = {}
time_spread = {}
time = {}
theory = {}
pdf = {}
readout = {}
ggH_higgsPt , VBFH_higgsPt , WH_higgsPt , ZH_higgsPt , ttH_higgsPt , ggZH_higgsPt =({} for i in range(6)) 
ggH_qcdScale , VBFH_qcdScale , WH_qcdScale , ZH_qcdScale , ttH_qcdScale , ggZH_qcdScale =({} for i in range(6)) 
ggH_pdf , VBFH_pdf , WH_pdf , ZH_pdf , ttH_pdf , ggZH_pdf =({} for i in range(6)) 
Nrechit_weight_file = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/Uncertainties/Nrechit_weight.txt'

cut_based_unc = [0.0921, 0.0852] # muon Eta reweighted
clustering_unc = [0.1506, 0.1047]

for m in mass:
    for ct in OLD_CTAU:
        for p in prod:
            if p == 'others':k = 'MC_ggH_'+str(m)+'_'+str(ct)   
            else:k = 'MC_'+p+'_'+str(m)+'_'+str(ct)         
            if p == 'others': key = 'MC_others_'+str(m)+'_'+str(ct)   
            else: key = k


            lumi[key] = [0.018]*4
            jetVeto[key] = [0.00068]*4
            muonVeto[key] = [0.045]*4
            rechitVeto[key] = [0.001]*4
            time[key] = [0.009]*4
            time_spread[key] = [0.028]*4
            clusterEff_unc[key]  = [0.035] * 4 #shifting Nrechit cut from 130 to 135
            cut_based_eff_unc[key]=[0.051]*4 #muon eta reweighted
            readout[key]=[0.01]*4 
            JES[key] = [0.084,0.083,0.042,0.041]
            pileup[key] = [0.008, 0.0110, 0.0110, 0.0110]
            

            ggH_higgsPt[key], VBFH_higgsPt[key], WH_higgsPt[key] , ZH_higgsPt[key] , ttH_higgsPt[key] , ggZH_higgsPt[key] = ([0.0]*8,)*6
            ggH_qcdScale[key], VBFH_qcdScale[key], WH_qcdScale[key] , ZH_qcdScale[key] , ttH_qcdScale[key] , ggZH_qcdScale[key] = ([0.0]*8,)*6
            ggH_pdf[key] , VBFH_pdf[key] , WH_pdf[key] , ZH_pdf[key] , ttH_pdf[key] , ggZH_pdf[key] = ([0.0]*4,)*6
            if 'ggH' in key: 
                ggH_higgsPt[key] = [0.205, 0.205,0.207,0.208, 0.133,0.133,0.134,0.134] #down*4/up*4
                ggH_qcdScale[key] = [0.067]*4+[0.046]*4 #down/up
                ggH_pdf[key] = [0.032]*4

            elif 'VBFH' in key: 
                VBFH_higgsPt[key] = [0.046, 0.159, 0.007, 0.006, 0.022, 0.032, 0.012, 0.010] #down/up
                VBFH_qcdScale[key] = [0.003]*4+[0.004]*4 #down/up
                VBFH_pdf[key] = [0.021]*4
            elif 'ggZH' in key:
                ggZH_higgsPt[key] = [0.194, 0.198, 0.208, 0.208, 0.120, 0.123, 0.134, 0.133] #down/up,ggZH
                ggZH_qcdScale[key] = [0.251]*4+[0.189]*4 #down/up
                ggZH_pdf[key] = [0.024]*4
#                 scaling[key] = [0.2]*4
            elif 'ZH' in key: 
                ZH_higgsPt[key] = [0.125, 0.057, 0.01, 0.017, 0.033,0.018, 0.006, 0.006] #down/up
                ZH_qcdScale[key] = [0.006]*4+[0.005]*4 #down/up
                ZH_pdf[key] = [0.019]*4
            elif 'WH' in key: 
                WH_higgsPt[key] = [0.071,0.040, 0.003, 0.054, 0.026, 0.024, 0.004, 0.018] #down/up
                WH_qcdScale[key] = [0.007]*4+[0.005]*4 #down/up
                WH_pdf[key] = [0.019]*4
            elif 'ttH' in key: 
                ttH_higgsPt[key] = [0.012, 0.148, 0.051, 0.028, 0.007, 0.066, 0.016, 0.010] #down/up
                ttH_qcdScale[key] = [0.092]*4+[0.058]*4 #down/up
                ttH_pdf[key] = [0.036]*4
            mc_stats[k] = []
            if p == 'ggZH': scaling[key] = [0.2]*4
            else: scaling[key] = [0,0,0,0]


sig_eff = [lumi, muonVeto, jetVeto,rechitVeto,time, time_spread, cut_based_eff_unc, clusterEff_unc,readout, JES,pileup, \
           ggH_higgsPt, VBFH_higgsPt, WH_higgsPt, ZH_higgsPt, ttH_higgsPt, ggZH_higgsPt,  \
           ggH_qcdScale, VBFH_qcdScale, WH_qcdScale, ZH_qcdScale, ttH_qcdScale, ggZH_qcdScale, \
           ggH_pdf,VBFH_pdf, WH_pdf, ZH_pdf, ttH_pdf, ggZH_pdf,\
           scaling, mc_stats]
sig_eff_name = ['lumi','muonVeto','jetVeto','rechitVeto','time','time_spread','cut_based_eff_unc', 'clusterEff_unc', 'readout', 'JES','pileup', \
                'ggH_higgsPt', 'VBFH_higgsPt', 'WH_higgsPt', 'ZH_higgsPt', 'ttH_higgsPt', 'ggZH_higgsPt',\
                'ggH_qcdScale', 'VBFH_qcdScale', 'WH_qcdScale', 'ZH_qcdScale', 'ttH_qcdScale', 'ggZH_qcdScale', \
                'ggH_pdf','VBFH_pdf', 'WH_pdf', 'ZH_pdf', 'ttH_pdf', 'ggZH_pdf',\
                'ggZH_reweight','mc_stats']


assert(len(sig_eff)==len(sig_eff_name))

In [87]:
signal_correction = (1-0.015)*(1-0.033)*(1-0.060)
print(signal_correction)

0.8953452999999999


# make datacard

In [91]:
import os
import math
import importlib
importlib.reload(sys.modules['helper'])
from helper import make_datacard, make_datacard_2sig, weight_calc
#methodB use oot vr and ABCD method
#methodC use the fit functions
N_RECHIT_CUTS = np.arange(60,220, 10)
# N_RECHIT_CUTS = [130]
ctaus = ['5','10','15','20','30','40', '50','60', '100', '125','150','200','300','500','600','700','800','900','1000', '2000','3000','4000', '5000', '6000','7000','8000','10000', '20000','30000','50000',\
         '100000', '200000', '300000', '500000', '1000000', '2000000', '3000000', '5000000', '6000000', '10000000'] #mm
UNBLIND = 2
# 0 use OOT region, 1 unblind ABC, 2 unblind ABCD
method = 2
fit_function = 'se' #se, me, sp
jetmet=0
datacard_version  = 'v1'

bkg_unc = [] #ABCD closure, sum of stats from two VR
bkg_unc_name = []
var = dphiMet_cluster
DPHI_CUT = 0.75
DPHI_CUTS = [0.75]
removeOverlap = 0
# print(bdtBkgEff_nrechit.shape)
bkg = []
print(N_RECHIT_CUTS)
for N_RECHIT_CUT in N_RECHIT_CUTS:
    if not N_RECHIT_CUT==130:continue
    for DPHI_CUT in DPHI_CUTS:
        bkg_unc = []
       
        
        k = 'data_intime_sr'
        if removeOverlap:
            a = np.count_nonzero(np.logical_and(np.logical_not(two_tag[k]), np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT)))
            b = np.count_nonzero(np.logical_and(np.logical_not(two_tag[k]), np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT)))
            c = np.count_nonzero(np.logical_and(np.logical_not(two_tag[k]), np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT)))
            d = np.count_nonzero(np.logical_and(np.logical_not(two_tag[k]), np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT))) 
        else:
            a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
            b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT))
            c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT))
            d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT))
        bkg_rate = np.array([a,b,c,a*c/b])
        if UNBLIND == 1:observation= np.array([a,b,c,a*c/b])
        else: observation= np.array([a,b,c,d])
        print(N_RECHIT_CUT, observation, bkg_unc)
        
        #####
        # a = c*c1*c2
        # b = c1* c
        # c = c
        # d = c2*c
        # start adding signal
        #####
#         /storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards_2tag/V1p17/v1/v137/v1/unblind/csc
        outDataCardsDir = '/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards_2tag/'+\
        ntupler_version+analyzer_version+'/'
#         if cut_based:outDataCardsDir += 'cut_based_'+cut_based_version+'/'
#         else: outDataCardsDir += bdt_name+'/'
        outDataCardsDir += datacard_version+'/'
        if UNBLIND == 0: outDataCardsDir += 'blind/'
        elif UNBLIND == 1: outDataCardsDir += 'unblindABC/'
        else:outDataCardsDir += 'unblind/'
        if removeOverlap: outDataCardsDir += '/csc_removeOverlap/'
        else: outDataCardsDir += '/csc/'
        if not os.path.isdir(outDataCardsDir):os.makedirs(outDataCardsDir)
        print(outDataCardsDir)
        sig_norm = []
        for m in mass:
            for ct in ctaus:
                dphi = str(DPHI_CUT).replace(".", "p")
                if len(prod) == 5: modelName = 'allProd_HToSSTo'+decay
                elif len(prod) ==6:  modelName = 'allProd_withggZH_HToSSTo'+decay
                else: modelName = ('_').join(prod)+'_HToSSTo'+decay
                modelName = modelName + '_mh125_mx'+str(m)+'_ctau'+str(ct)+'mm_nRechit'+str(N_RECHIT_CUT)+'dPhiCluster'+dphi
                
                signal_rate = {}
                mc_stat_unc = {}
                gmn = {}
                sig_unc = {}
                ctf = int(ct)
                ct_list = 10**int(math.log10(ctf))
                if ctf < OLD_CTAU[0]: ct_list = [OLD_CTAU[0]]
                elif ctf>OLD_CTAU[-1]: ct_list = [OLD_CTAU[-1]]
                elif ct_list == int(ct): ct_list = [int(ct)]
                else:ct_list = [ct_list,ct_list*10]
                for p in prod:                    
                    signal_rate[p] = np.zeros((4,))
                    mc_stat_unc[p] = np.zeros((4,))
                    gmn[p] = np.zeros((4,))
                    sig_unc[p] = []
                    if p == 'others':
                        for i in range(4):
                            signal_rate[p][i] = signal_rate['ggH'][i]*0.52
                            mc_stat_unc[p][i] = (mc_stat_unc['ggH'][i]*signal_rate[p][i])**2
                    else:
                        for i, ct0 in enumerate(ct_list):
                            k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)
                            T = tree[k]
                            if np.count_nonzero(sel_ev[k])==0:continue
#                             gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                            gLLP_ctau = T.array('gLLP_ctau')[sel_ev[k]][bdt_sel[k]]

                            weight_ctau = weight_calc(gLLP_ctau, int(ct)/10, int(ct0)/10) # convert everything to cm
                            gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                            if len(ct_list) == 1:weight_cond = gLLP_ctau >= 0
                            else:
                                if i == 0 : weight_cond = gLLP_ctau<int(ct_list[0]/2)
                                else: weight_cond = gLLP_ctau>=int(ct_list[0]/2)
                            w = weight[k]*weight_ctau
                            if removeOverlap: cond = np.logical_and(np.logical_not(two_tag[k]), np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond))
                            else:cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond)
                            signal_rate[p][0]+=np.sum(w[cond])
                            mc_stat_unc[p][0]+=np.sum(w[cond]**2)
                            gmn[p][0] += len(w[cond])
                            if removeOverlap: cond = np.logical_and(np.logical_not(two_tag[k]), np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond))
                            else:cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond)
                            signal_rate[p][1]+=np.sum(w[cond])
                            mc_stat_unc[p][1]+=np.sum(w[cond]**2)
                            gmn[p][1] += len(w[cond])
                            if removeOverlap: cond = np.logical_and(np.logical_not(two_tag[k]), np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond))
                            else:cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond)
                            signal_rate[p][2]+=np.sum(w[cond])
                            mc_stat_unc[p][2]+=np.sum(w[cond]**2)
                            gmn[p][2] += len(w[cond])
                            if removeOverlap: cond = np.logical_and(np.logical_not(two_tag[k]), np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond))
                            else:cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond)
                            signal_rate[p][3]+=np.sum(w[cond])
                            mc_stat_unc[p][3]+=np.sum(w[cond]**2)
                            gmn[p][3] += len(w[cond])
                            # apply the corrections
                            for j in [0,1,2,3]:
                                if signal_rate[p][j]<0.0:
                                    signal_rate[p][j]=0.0
                                    mc_stat_unc[p][j]=0.0
                                    gmn[p][j] = 0.0
                                else:
                                    signal_rate[p][j] *= signal_correction

#                     mc_stat_unc[p] = list(np.nan_to_num(np.sqrt(mc_stat_unc[p])/signal_rate[p])) #convert absolute to relative uncertainty
                    
                    mc_stat_unc[p] = np.nan_to_num(signal_rate[p]/np.sqrt(mc_stat_unc[p]))**2 #gmn
                    mc_stat_unc[p][mc_stat_unc[p] == np.inf] = 0.0
                    
                    mc_stat_unc[p] = list(mc_stat_unc[p])
                    mc_stat_unc[p] = list(gmn[p])
                    if p == 'others': k = k.replace("ggH", "others")
                    for j, ele in enumerate(sig_eff):# go through each uncertainty
                        if j == len(sig_eff)-1: # if mc_stats
                            sig_unc[p].append(mc_stat_unc[p])
#                         elif sig_eff_name[j] in simulation_name:
# #                             print(type(ele[k]))
#                             sig_unc[p].append(np.array(ele[k])*0.5)
                        else:
                            sig_unc[p].append(ele[k])
                   
                
                norm = np.sum(signal_rate['ggH'])/4
#                 if norm == 0.0:continue
#                 for k,v in signal_rate.items():signal_rate[k] = v/norm
#                 print(modelName, norm, signal_rate['ggH'], mc_stat_unc['ggH'])
                make_datacard_2tag(outDataCardsDir, modelName, signal_rate, norm, bkg_rate, observation, \
                                  bkg_unc, bkg_unc_name, sig_unc, sig_eff_name, 'd','')
                

[ 60  70  80  90 100 110 120 130 140 150 160 170 180 190 200 210]
130 [ 3 96 47  3] []
/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards_2tag/V1p17/v2/v138//v1/unblind//csc/


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:144: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:144: RuntimeWarning: overflow encountered in square


# unblind

In [ ]:
import math

In [44]:
k = 'data_intime_sr'

n_ev = 5000
var = np.abs(dphiMet_cluster[k])
DPHI_CUT = 0.75
print("Nrechits, A, B, C, pred, D")
for N_RECHIT_CUT in np.arange(60,240,10):
    a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var>=DPHI_CUT))
    b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var>=DPHI_CUT))
    c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var<DPHI_CUT))
    d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var<DPHI_CUT))
    pred = c/b*a
    unc_pred = (1./c + 1./b + 1./a)**0.5*(c/b*a)

    print(N_RECHIT_CUT, '\t',a,'\t',b,'\t',c,'\t',d,'\t', round(c/b*a, 2), '\t',\
          round( (1./c + 1./b + 1./a)**0.5*(c/b*a), 2))


Nrechits, A, B, C, pred, D
60 	 47 	 52 	 20 	 30 	 18.08 	 5.44
70 	 22 	 77 	 32 	 18 	 9.14 	 2.74
80 	 11 	 88 	 38 	 12 	 4.75 	 1.7
90 	 9 	 90 	 42 	 8 	 4.2 	 1.6
100 	 3 	 96 	 42 	 8 	 1.31 	 0.8
110 	 3 	 96 	 42 	 8 	 1.31 	 0.8
120 	 3 	 96 	 43 	 7 	 1.34 	 0.81
130 	 3 	 96 	 47 	 3 	 1.47 	 0.89
140 	 2 	 97 	 47 	 3 	 0.97 	 0.71
150 	 2 	 97 	 47 	 3 	 0.97 	 0.71
160 	 2 	 97 	 48 	 2 	 0.99 	 0.72
170 	 1 	 98 	 49 	 1 	 0.5 	 0.51


ZeroDivisionError: float division by zero